In [11]:
import torch
import torch.autograd as agrad
import numpy as np
import matplotlib.pyplot as plt
from operators.stokes_operator import StokesOperator


# Example of simple boundary
N = 100
dt = np.pi * 2 / N
t  = torch.arange(0,N) * dt
z = torch.exp(1j * t) + 0.1 * torch.exp(5j * t)
dz = 1j * torch.exp(1j * t) + 0.1 * 5j * torch.exp(5j * t);
ddz = 1j * 1j *torch.exp(1j * t) + 0.1 * 5j * 5j * torch.exp(5j * t);
w = torch.ones(N,) * dt

# Stokes operator
A = StokesOperator(z, dz, ddz, w)

# Network
a = torch.Tensor([1.5])
a.requires_grad_(True)

# Target density
v0 = torch.exp(2j * t)[None, None, :]


# Network
def net(a):
    v = torch.exp(1j * a * t)[None, None, :]
    v.requires_grad_(True)
    for i in range(5):
        mse = torch.norm(A(v) - v0)**2
        v = v - 0.1 * agrad.grad(mse, v, retain_graph=True, create_graph=True)[0]
    return v

# Training
grads = []
alist = torch.linspace(0.5,3,20)
for ai in alist:
    ai.requires_grad_(True)
    loss = torch.norm(net(ai) - v0) ** 2
    
    ai.retain_grad()
    loss.backward()
    
    grads.append(ai.grad.numpy())

    
a_train = []
agrad_train = []
for i in range(30):
    loss = torch.norm(net(a) - v0) ** 2 * dt
    a.retain_grad()
    loss.backward()    
    
    a_train.append(a.detach())
    agrad_train.append(a.grad.detach())
    
    a = a - 0.01*a.grad
    a.grad = None
    

v = net(a).detach()
 
x, y = torch.real(z), torch.imag(z)
ux, uy = torch.real(v.squeeze()), torch.imag(v.squeeze())
ux0, uy0 = torch.real(v0.squeeze()), torch.imag(v0.squeeze())


In [12]:
    
plt.plot(alist, grads * dt)
plt.scatter(a_train, agrad_train)
plt.plot(alist, 4 * np.cumsum(np.array(grads))*dt)
plt.plot(alist, 0*alist)


plt.figure()

plt.plot(x, y)
plt.quiver(x, y, ux0, uy0, color = 'red', scale=10)
plt.quiver(x, y, ux, uy)
plt.axis("equal")

TypeError: can't multiply sequence by non-int of type 'float'